




































































































































































































































































































































































































































































































































































































###### SELL BILL QNTY WISE

In [4]:
import mysql.connector


In [3]:
import os
import win32com.client
import subprocess


def convert_excel_to_pdf(input_excel_path, output_pdf_path):
    subprocess.call(["taskkill", "/f", "/im", "EXCEL.EXE"])
    excel = win32com.client.Dispatch("Excel.Application")
    excel.Visible = False
    
    try:
        # Open the Excel file
        workbook = excel.Workbooks.Open(os.path.abspath(input_excel_path))
        
        # Save as PDF
        workbook.ExportAsFixedFormat(0, os.path.abspath(output_pdf_path))
        
        # Close the workbook
        workbook.Close()
    finally:
        # Quit Excel application
        excel.Quit()

if __name__ == "__main__":
    input_excel_path = r"C:\Users\AMARJEET\Desktop\python_bill\SELL_BILL\EXCEL\sk"+"0956"+".xlsx"
    output_pdf_path = r"C:\Users\AMARJEET\Desktop\python_bill\SELL_BILL\EXCEL\sk"+"tesngpfg"+".pdf"

    convert_excel_to_pdf(input_excel_path, output_pdf_path)


In [ ]:
# import os
import shutil
import re
from openpyxl import load_workbook
from datetime import datetime
from win32com import client
import num2words

import os
import psutil
from time import sleep

import pandas as pd
import pdfkit



while(True):
    res =False
    while res ==False:
    
        invoice_no=input("enter current invoice number")
        pat = re.compile(r"[0-9]+")
    # Checks whether the whole string matches the re.pattern or not
        if re.fullmatch(pat, invoice_no):
            res=True
        else:
            print("not valid invoice number")
        





    source = r"C:\Users\AMARJEET\Desktop\python_bill\formate.xlsx"
    destination= r"C:\Users\AMARJEET\Desktop\python_bill\SELL_BILL\EXCEL\sk"+invoice_no+".xlsx"
    file_loc=r"C:\Users\AMARJEET\Desktop\python_bill\CUSTOMER.txt"
    
    

    dest = shutil.copyfile(source, destination)

#load excel file
    workbook = load_workbook(filename=destination)
 
    #open workbook
    sheet = workbook.active


    cos_file=open(file_loc)
    cus_details=cos_file.readlines()
    cus_find=False
    while cus_find==False:
        name=input("enter the customer name")
        list_len=len(cus_details)
        for i in cus_details:
            if (i.split('>')[0]==name):
                add=i.split('>')[1]
                cus_find=True

    
    
 
 # printing result
#print("Does date match format? : " + str(res))


    res =False
    while res==False:
   
        date=input("enter todays date")
        format = "%d-%m-%Y"
        try:
            res = bool(datetime.strptime(date, format))
        except ValueError:
            res = False
    
    
    sheet["H3"]=date
    sheet["H5"]=invoice_no
    cell_no=13
    add_list=add.split("|")
    for i in add_list:
        cell_str="B"+str(cell_no)
        sheet[cell_str]=i
        cell_str="E"+str(cell_no)
        sheet[cell_str]=i
        cell_no=cell_no+1
    
    sum_total=0
    status=False
    discription_details=[]
    HSN_dis={}
    GST_HSN={}
    
    pur_invoice_no=input("enter purchase bill no")
    pur_destination= r"C:\Users\AMARJEET\Desktop\python_bill\AUDIT\Audit"+pur_invoice_no+".xlsx"
    
        #load excel file
    workbook_pur = load_workbook(filename=pur_destination) 
    #open workbook
    sheet_pur = workbook_pur.active
    print(sheet_pur)
    while status ==False:
        change_bill=int(input("press 1 to change_purchase_bill"))
        if change_bill==1:
            workbook_pur.save(filename=pur_destination)
            pur_invoice_no=input("enter purchase bill no")
            pur_destination= r"C:\Users\AMARJEET\Desktop\python_bill\AUDIT\Audit"+pur_invoice_no+".xlsx"
    
        #load excel file
            workbook_pur = load_workbook(filename=pur_destination)
 
        #open workbook
            sheet_pur = workbook_pur.active

        temp_list=[]
        pur_sl=int(input("enter serial no of product according to purchase bill"))
        cell_str="B"+str(5+pur_sl)
        try:
            discription=sheet_pur[cell_str].value
            print("product_discription:",discription)
        except Exception as Ex:
            print("Exception Occured during reading discription:",Ex)
            
        
        temp_list.append(discription)
        
        cell_str="G"+str(5+pur_sl)
        hsn=int(sheet_pur[cell_str].value)
        temp_list.append(hsn)
        if hsn in HSN_dis.keys():
            cc="sdddddd"
        else:
            HSN_dis[hsn]=0
        
        
        
        cell_str="V"+str(5+pur_sl)
        qty_rem=int(sheet_pur[cell_str].value)
        if qty_rem ==0:
            print("this item is not in stock")
            continue
        print("quantity in stock for this product is",qty_rem)
        full_qty=int(input("press 1 to use full qty"))
        if full_qty==1:
            temp_quantity=qty_rem
        
        else:
            temp_quantity=int(input("quantity"))
        if temp_quantity>qty_rem:
            print("entered quantity is not avilable in stock")
            continue
        temp_list.append(temp_quantity)
        cell_str="I"+str(5+pur_sl)
        rate=round(float(sheet_pur[cell_str].value),2)
        
        
        cell_str="J"+str(5+pur_sl)
        dis=round(float(sheet_pur[cell_str].value),2)
        
       
        rate=round((rate+(0.032*rate)),2)
        temp_list.append(rate)
        print("optimum rate for product is ",rate)
        quantity=temp_quantity
        HSN_dis[hsn]=HSN_dis[hsn]+((quantity*rate)-(quantity*rate*dis/100))
        temp_list.append(dis)
        temp_list.append((quantity*rate)-(quantity*rate*dis/100))
        amount=(quantity*rate)-(quantity*rate*dis/100)
        
        
        cell_str="S"+str(5+pur_sl)
        gst=int(sheet_pur[cell_str].value)
        if hsn in GST_HSN.keys():
            cc="sdddddd"
        else:
            GST_HSN[hsn]=gst
        
        #gst=int(input("GST_rate"))
        sum_total=sum_total+(quantity*rate)+(quantity*rate*gst/100)
        temp_list.append(round((quantity*rate)+(quantity*rate*gst/100),2))
        discription_details.append(temp_list)
        print("total amount till now incluing gst",sum_total)
        
        #inserting sell bill_no
        cell_str="L"+str(5+pur_sl)
        try:
            sell_bill_no=sheet_pur[cell_str].value
            sell_bill_no=sell_bill_no+'|'+str(invoice_no)
        except TypeError:
            sell_bill_no=str(invoice_no)
        sheet_pur[cell_str]=sell_bill_no
        #inserting sell_qty_wrt_bill
        cell_str="M"+str(5+pur_sl)
        try:
            sell_qty_no=sheet_pur[cell_str].value
            sell_qty_no=sell_qty_no+'|'+str(quantity)
        except TypeError:
            sell_qty_no=str(quantity)
        sheet_pur[cell_str]=sell_qty_no
        
        
        #inserting total_quantity_selled in audit
        
        cell_str="O"+str(5+pur_sl)
        try:
            total_selled_qntity=int(sheet_pur[cell_str].value)
        except TypeError:
            total_selled_qntity=0;
        
        total_selled_qntity=total_selled_qntity+quantity
        sheet_pur[cell_str]=str(total_selled_qntity)
        
        #inserting total selled qntity
        cell_str="O"+str(5+pur_sl)
        sheet_pur[cell_str]=total_selled_qntity
        cell_str="U"+str(5+pur_sl)
        sheet_pur[cell_str]=total_selled_qntity
        
        
        #inserting quantity in stock
        cell_str="H"+str(5+pur_sl)
        pur_total_qntity=int(sheet_pur[cell_str].value)
        stock_qnty=pur_total_qntity-total_selled_qntity
        cell_str="V"+str(5+pur_sl)
        sheet_pur[cell_str]=stock_qnty
        
        #inserting sell amount
        
        cell_str="X"+str(5+pur_sl)
        try:
            total_selled_amount=int(sheet_pur[cell_str].value)
        except TypeError:
            total_selled_amount=0
        total_selled_amount=total_selled_amount+amount
        sheet_pur[cell_str]=total_selled_amount
        
        cell_str="P"+str(5+pur_sl)
        sheet_pur[cell_str]= round( (float(total_selled_amount)/float(total_selled_qntity)),2)
   
        #inserting gst__received
        
        cell_str="Z"+str(5+pur_sl)
        try:
            gst_amount=float(round(sheet_pur[cell_str].value))
        except TypeError:
            gst_amount=0
        gst_amount=gst_amount+round((amount*gst/100),2)
        sheet_pur[cell_str]=str(gst_amount)
        
        #inserting gst__balance
        
        
        cell_str="Y"+str(5+pur_sl)
        try:
            gst_recv=int(sheet_pur[cell_str].value)
        except TypeError:
            gst_recv=0
        gst_balance=gst_amount-gst_recv
        cell_str="AA"+str(5+pur_sl)
        sheet_pur[cell_str]=str(gst_balance)
        
        
        x=int(input("press 1 to add more"))
        if(x==1):
              status=False
        else:
              status=True
    num=21
    count=0
    s_t=0
    print(discription_details)
    for i in discription_details:
        count=count+1
        cell_str="A"+str(num)
        sheet[cell_str]=count
        cell_str="B"+str(num)
        sheet[cell_str]=i[0]
        cell_str="E"+str(num)
        sheet[cell_str]=i[1]
        cell_str="F"+str(num)
        sheet[cell_str]=i[2]
        cell_str="G"+str(num)
        sheet[cell_str]=i[3]
        cell_str="H"+str(num)
        sheet[cell_str]=i[4]
        cell_str="I"+str(num)
        sheet[cell_str]=i[5]
        s_t=s_t+i[-2]
        num=num+1
    
    sheet["H27"]=s_t
    
    
    count=37
    tax_sum=0
    for key, value_1 in HSN_dis.items():
    
        cell_str="B"+str(count)
        sheet[cell_str]=key
        sheet.cell(count,3).value=value_1
    #cell_str="D"+str(count)
    #sheet[cell_str]=value_1
        cell_str="E"+str(count)
        sheet[cell_str]=(GST_HSN[key]/2)/100
        cell_str="F"+str(count)
        sheet[cell_str]=(GST_HSN[key]/2)*value_1/100
        cell_str="G"+str(count)
        sheet[cell_str]=(GST_HSN[key]/2)/100
        cell_str="H"+str(count)
        sheet[cell_str]=(GST_HSN[key]/2)*value_1/100
        cell_str="I"+str(count)
        sheet[cell_str]=(GST_HSN[key])*value_1/100
        tax_sum=tax_sum+(GST_HSN[key])*value_1/100
        count=count+1
    sheet["J41"]=tax_sum
    sheet["C42"]=num2words.num2words(round(tax_sum,2), lang='en_IN')+"ONLY"
    sheet["H28"]=tax_sum/2
    sheet["H29"]=tax_sum/2
    sheet["H30"]=s_t+tax_sum
    
    
    sum_t=s_t+tax_sum
    r_total=round(sum_t,0)
    sheet['H31']=r_total-sum_t
    sheet['H32']=r_total
    print("######################################################################")
    print("total ammount after Rounding off :",r_total)
    sheet["C34"]=num2words.num2words(r_total, lang='en_IN')+"ONLY"
    print (num2words.num2words(r_total, lang='en_IN')+"ONLY")

#save the file
    workbook_pur.save(filename=pur_destination)
    workbook.save(filename=destination)
    #workbook.close()

    pdf_path=r"C:\Users\AMARJEET\Desktop\python_bill\SELL_BILL\PDF\sk"+invoice_no
    pdf_path1=r"C:\Users\AMARJEET\Desktop\python_bill\SELL_BILL\PDF\SEP23\sk"+invoice_no
# Open  Excel
    convert_excel_to_pdf(destination,pdf_path)
    # Read Excel File
  #  work_sheets.close()
    #df = pd.read_excel(destination)
    #df.to_html("file.html")
    #pdfkit.from_file("file.html",pdf_path)
   # for i in range(1):    
    #for p in psutil.process_iter(): #Close Acrobat after printing the PDF EKTA3509
     #        if 'Excel' in str(p):
      #          p.kill()                                                                                                                                                                                                 

    
    

enter current invoice number232324
enter the customer nameCM1


# SELL BILL

In [5]:
import os
import shutil
import re
from openpyxl import load_workbook
from datetime import datetime
from win32com import client
import num2words
while(True):
    res =False
    while res ==False:
    
        invoice_no=input("enter current invoice number")
        pat = re.compile(r"[0-9]+")
    # Checks whether the whole string matches the re.pattern or not
        if re.fullmatch(pat, invoice_no):
            res=True
        else:
            print("not valid invoice number")
        





    source = r"C:\Users\HP\Desktop\python_bill\formate.xlsx"
    destination= r"C:\Users\HP\Desktop\python_bill\SELL_BILL\EXCEL\sk"+invoice_no+".xlsx"
    file_loc=r"C:\Users\HP\Desktop\python_bill\CUSTOMER.txt"


    dest = shutil.copyfile(source, destination)

#load excel file
    workbook = load_workbook(filename=destination)
 
    #open workbook
    sheet = workbook.active


    cos_file=open(file_loc)
    cus_details=cos_file.readlines()
    cus_find=False
    while cus_find==False:
        name=input("enter the customer name")
        list_len=len(cus_details)
        for i in cus_details:
            if (i.split('>')[0]==name):
                add=i.split('>')[1]
                cus_find=True

    
    
 
 # printing result
#print("Does date match format? : " + str(res))


    res =False
    while res==False:
   
        date=input("enter todays date")
        format = "%d-%m-%Y"
        try:
            res = bool(datetime.strptime(date, format))
        except ValueError:
            res = False
    
    
    sheet["H3"]=date
    sheet["H5"]=invoice_no
    cell_no=13
    add_list=add.split("|")
    for i in add_list:
        cell_str="B"+str(cell_no)
        sheet[cell_str]=i
        cell_str="E"+str(cell_no)
        sheet[cell_str]=i
        cell_no=cell_no+1
    
    sum_total=0
    status=False
    discription_details=[]
    HSN_dis={}
    while status ==False:
        temp_list=[]
        temp_list.append(input("discription"))
        hsn=int(input("HSN"))
        temp_list.append(hsn)
        if hsn in HSN_dis.keys():
            cc="sdddddd"
        else:
            HSN_dis[hsn]=0
    
        temp_quantity=int(input("quantity"))
        unit=input("D or P")
        rate=float(input("rate"))
        dis=float(input("float"))
        if unit =='D':
            temp_list.append(temp_quantity*12)
            quantity=temp_quantity*12
            temp_list.append(round((rate/12)+0.04*(rate/12),2))
            rate=round(((rate/12)+0.04*(rate/12)),2)
        else:
        
            temp_list.append(temp_quantity)
            quantity=temp_quantity
            temp_list.append(round(rate+(0.04*rate),2))
            rate=round((rate+(0.04*rate)),2)
        HSN_dis[hsn]=HSN_dis[hsn]+((quantity*rate)-(quantity*rate*dis/100))
        temp_list.append(dis)
        temp_list.append((quantity*rate)-(quantity*rate*dis/100))
        gst=int(input("GST_rate"))
        sum_total=sum_total+(quantity*rate)+(quantity*rate*gst/100)
        temp_list.append(round((quantity*rate)+(quantity*rate*gst/100),2))
        discription_details.append(temp_list)
        print("total amount till now incluing gst",sum_total)
        x=int(input("press 1 to add more"))
        if(x==1):
              status=False
        else:
              status=True
    num=21
    count=0
    s_t=0
    for i in discription_details:
        count=count+1
        cell_str="A"+str(num)
        sheet[cell_str]=count
        cell_str="B"+str(num)
        sheet[cell_str]=i[0]
        cell_str="E"+str(num)
        sheet[cell_str]=i[1]
        cell_str="F"+str(num)
        sheet[cell_str]=i[2]
        cell_str="G"+str(num)
        sheet[cell_str]=i[3]
        cell_str="H"+str(num)
        sheet[cell_str]=i[4]
        cell_str="I"+str(num)
        sheet[cell_str]=i[5]
        s_t=s_t+i[-2]
        num=num+1
    
    sheet["H27"]=s_t
    sheet["H28"]=(s_t*gst/100)/2
    sheet["H29"]=(s_t*gst/100)/2
    sheet["H30"]=s_t+(s_t*gst/100)
    
    sum_t=s_t+(s_t*gst/100)
    r_total=round(sum_t,0)
    sheet['H31']=r_total-sum_t
    sheet['H32']=r_total
    sheet["C34"]=num2words.num2words(r_total, lang='en_IN')+"ONLY"
    count=37
    tax_sum=0
    for key, value_1 in HSN_dis.items():
    
        cell_str="B"+str(count)
        sheet[cell_str]=key
        sheet.cell(count,3).value=value_1
    #cell_str="D"+str(count)
    #sheet[cell_str]=value_1
        cell_str="E"+str(count)
        sheet[cell_str]=GST_HSN[key]/100*0.5
        cell_str="F"+str(count)
        sheet[cell_str]=GST_HSN[key]*value_1/100*0.5
        cell_str="G"+str(count)
        sheet[cell_str]=GST_HSN[key]/100*0.5
        cell_str="H"+str(count)
        sheet[cell_str]=GST_HSN[key]*value_1/100*0.5
        cell_str="I"+str(count)
        sheet[cell_str]=GST_HSN[key]*value_1/100
        tax_sum=tax_sum+GST_HSN[key]*value_1/100
        count=count+1
    sheet["J41"]=tax_sum
    sheet["C42"]=num2words.num2words(round(tax_sum,2), lang='en_IN')+"ONLY"
    

#save the file
    workbook.save(filename=destination)

    pdf_path=r"C:\Users\HP\Desktop\python_bill\SELL_BILL\PDF\sk"+invoice_no
# Open Microsoft Excel
   
    
    

enter current invoice number0746
enter the customer nameSAMAY
enter todays date09-04-2023
discription';L;'L56
HSNPIP


ValueError: invalid literal for int() with base 10: 'PIP'

# PRCHASE BILL

In [2]:
import os
import shutil
import re
from openpyxl import load_workbook
from datetime import datetime
from win32com import client
import num2words
while(True):
    flag=int(input("enter 1 to exit"))
    if flag==1:
        break
    
    invoice_no=input("enter PURCHASE invoice number")
        





    source = r"C:\Users\HP\Desktop\python_bill\AUDIT_FORMATE.xlsx"
    destination= r"C:\Users\HP\Desktop\python_bill\AUDIT\Audit"+invoice_no+".xlsx"
    file_loc=r"C:\Users\HP\Desktop\python_bill\CUSTOMER.txt"


    dest = shutil.copyfile(source, destination)

#load excel file
    workbook = load_workbook(filename=destination)
 
    #open workbook
    sheet = workbook.active
    vendor=input("enter vendor name")
    


    res=False
    while res==False:
   
        date=input("enter Purchase bill date")
        format = "%d-%m-%Y"
        try:
            res = bool(datetime.strptime(date, format))
        except ValueError:
            res = False
    
    
    sheet["C2"]=vendor
    sheet["M2"]=vendor
    sheet["U2"]=vendor
    sheet["I3"]=invoice_no
    sheet["M3"]=invoice_no
    sheet["U3"]=invoice_no
    sheet["C3"]=date
    product_entry=1
    unifrom_GST=int(input("press 1 if GST is uniform"))
    if unifrom_GST==1:
        GST_RATE=round(float(input("enter GST rate ")))
    unifrom_HSN=int(input("press 1 if HSN is uniform"))
    if unifrom_HSN==1:
        HSN=int(input("enter HSN"))
    
    while product_entry ==1:
        slno=int(input("enter sl no as of purchased bill"))
        cell_str="A"+str(5+slno)
        sheet[cell_str]=slno
        cell_str="K"+str(5+slno)
        sheet[cell_str]=slno
        cell_str="R"+str(5+slno)
        sheet[cell_str]=slno
    
        description=input("enter the discription")
        cell_str="B"+str(5+slno)
        sheet[cell_str]=description
        if unifrom_HSN!=1:
            HSN=int(input("enter HSN CODE"))
        cell_str="G"+str(5+slno)
        sheet[cell_str]=HSN
        D_P=input("QUANTITY UNIT D or P")
        
        qty_purchased=int(input("enter the quantity purchased"))
        rate_purchased=round(float(input("enter the rate purchased")),2)
        
        if D_P=='D':
            qty_purchased=qty_purchased*12
            rate_purchased=round((rate_purchased/12),2)
        disc=round(float(input("enter the discunt")),2)
        cell_str="H"+str(5+slno)
        sheet[cell_str]=qty_purchased
        cell_str="T"+str(5+slno)
        sheet[cell_str]=qty_purchased
        
        cell_str="I"+str(5+slno)
        sheet[cell_str]=rate_purchased
        
        cell_str="J"+str(5+slno)
        sheet[cell_str]=disc
        if unifrom_GST !=1:
            GST_RATE=round(float(input("enter GST rate ")))
            
        cell_str="S"+str(5+slno)
        sheet[cell_str]=GST_RATE
        
        total_amount=round(float(input("enter total amount")),2)
        
        cell_str="W"+str(5+slno)
        sheet[cell_str]=total_amount
        
        #gst_amount=round(float(input("enter gst_amount in bill")),2)
        #cell_str="Y"+str(5+slno)
        #sheet[cell_str]=gst_amount
        
        
        
        
        
        
            
            
        product_entry=int(input("press 1 to enter a product"))
    
    
   
    #save the file
    workbook.save(filename=destination)
    
                  

        
        
        

enter 1 to exit0
enter PURCHASE invoice numberEKTA695
enter vendor nameEKTA AGENCIES
enter Purchase bill date20-04-2023
press 1 if GST is uniform1
enter GST rate 12
press 1 if HSN is uniform1
enter HSN48202000
enter sl no as of purchased bill1
enter the discriptionC32 17PG CRAZY A/4 BOOK
QUANTITY UNIT D or PP
enter the quantity purchased72
enter the rate purchased47.410
enter the discunt0
enter total amount3413.520
press 1 to enter a product0
enter 1 to exit0
enter PURCHASE invoice numberSAMBHAVA000065
enter vendor nameSAMBHAV STATIONARY
enter Purchase bill date17-04-2023
press 1 if GST is uniform1
enter GST rate 18
press 1 if HSN is uniform1
enter HSN7310
enter sl no as of purchased bill1
enter the discriptionMAX GEOM BOX
QUANTITY UNIT D or PP
enter the quantity purchased200
enter the rate purchased50.850
enter the discunt5
enter total amount10170
press 1 to enter a product0
enter 1 to exit0
enter PURCHASE invoice numberJIGAR00269
enter vendor nameJIGAR AGENCIES
enter Purchase bill da

# PURCHASE BILL 2.0


In [4]:
import os
import shutil
import re
from openpyxl import load_workbook
from datetime import datetime
from win32com import client
import num2words
while(True):
    flag=int(input("enter 1 to exit"))
    if flag==1:
        break
    
    invoice_no=input("enter PURCHASE invoice number")
        





    source = r"C:\Users\HP\Desktop\python_bill\AUDIT_FORMATE.xlsx"
    destination= r"C:\Users\HP\Desktop\python_bill\AUDIT\Audit"+invoice_no+".xlsx"
    file_loc=r"C:\Users\HP\Desktop\python_bill\CUSTOMER.txt"


    dest = shutil.copyfile(source, destination)

    #load excel file
    workbook = load_workbook(filename=destination)
 
    #open workbook
    sheet = workbook.active
    vendor=input("enter vendor name")
    


    res=False
    while res==False:
   
        date=input("enter Purchase bill date")
        format = "%d-%m-%Y"
        try:
            res = bool(datetime.strptime(date, format))
        except ValueError:
            res = False
    
    
    sheet["C2"]=vendor
    sheet["M2"]=vendor
    sheet["U2"]=vendor
    sheet["I3"]=invoice_no
    sheet["M3"]=invoice_no
    sheet["U3"]=invoice_no
    sheet["C3"]=date
    product_entry=1
    unifrom_GST=int(input("press 1 if GST is uniform"))
    if unifrom_GST==1:
        GST_RATE=round(float(input("enter GST rate ")))
    unifrom_HSN=int(input("press 1 if HSN is uniform"))
    if unifrom_HSN==1:
        HSN=int(input("enter HSN"))
    
    while product_entry ==1:
        slno=int(input("enter sl no as of purchased bill"))
        
        cell_str="A"+str(5+slno)
        sheet[cell_str]=slno
        cell_str="K"+str(5+slno)
        sheet[cell_str]=slno
        cell_str="R"+str(5+slno)
        sheet[cell_str]=slno
    
        description=input("enter the discription")
        cell_str="B"+str(5+slno)
        sheet[cell_str]=description
        if unifrom_HSN!=1:
            HSN=int(input("enter HSN CODE"))
        cell_str="G"+str(5+slno)
        sheet[cell_str]=HSN
        D_P=input("QUANTITY UNIT D or P")
        
        qty_purchased=int(input("enter the quantity purchased"))
        rate_purchased=round(float(input("enter the rate purchased")),2)
        
        if D_P=='D':
            qty_purchased=qty_purchased*12
            rate_purchased=round((rate_purchased/12),2)
        disc=round(float(input("enter the discunt")),2)
        cell_str="H"+str(5+slno)
        sheet[cell_str]=qty_purchased
        cell_str="T"+str(5+slno)
        sheet[cell_str]=qty_purchased
        cell_str="V"+str(5+slno)
        sheet[cell_str]=qty_purchased
        
        
        
        cell_str="I"+str(5+slno)
        sheet[cell_str]=rate_purchased
        
        cell_str="J"+str(5+slno)
        sheet[cell_str]=disc
        if unifrom_GST !=1:
            GST_RATE=round(float(input("enter GST rate ")))
            
        cell_str="S"+str(5+slno)
        sheet[cell_str]=GST_RATE
        
        total_amount=round(float(input("enter total amount")),2)
        
        cell_str="W"+str(5+slno)
        sheet[cell_str]=total_amount
        
        cell_str="Y"+str(5+slno)
        sheet[cell_str]=round(float(total_amount*GST_RATE/100),2)
        
        cell_str="U"+str(5+slno)
        sheet[cell_str]=0
        
        
        
        
        #gst_amount=round(float(input("enter gst_amount in bill")),2)
        #cell_str="Y"+str(5+slno)
        #sheet[cell_str]=gst_amount
        
        
        
        
        
        
            
            
        product_entry=int(input("press 1 to enter a product"))
    
    
   
    #save the file
    workbook.save(filename=destination)


enter 1 to exit0
enter PURCHASE invoice number0746
enter vendor nameSAMAY
enter Purchase bill date09-04-2023
press 1 if GST is uniform1
enter GST rate 12
press 1 if HSN is uniformHJUFYFYG


ValueError: invalid literal for int() with base 10: 'HJUFYFYG'

# CHECK BILLS
E

In [ ]:
import os
import shutil
import re
from openpyxl import load_workbook
from datetime import datetime
from win32com import client
import num2words
import psutil
from time import sleep
while(True):
    res =False
    while res ==False:
    
        invoice_no=input("enter current invoice number")
        pat = re.compile(r"[0-9]+")
    # Checks whether the whole string matches the re.pattern or not
        if re.fullmatch(pat, invoice_no):
            res=True
        else:
            print("not valid invoice number")
        





    source = r"C:\Users\AMARJEET\Desktop\python_bill\formate.xlsx"
    destination= r"C:\Users\AMARJEET\Desktop\python_bill\SELL_BILL\EXCEL\sk"+invoice_no+".xlsx"
    file_loc=r"C:\Users\AMARJEET\Desktop\python_bill\CUSTOMER.txt"
    
    

    dest = shutil.copyfile(source, destination)

#load excel file
    workbook = load_workbook(filename=destination)
 
    #open workbook
    sheet = workbook.active


    cos_file=open(file_loc)
    cus_details=cos_file.readlines()
    cus_find=False
    while cus_find==False:
        name=input("enter the customer name")
        list_len=len(cus_details)
        for i in cus_details:
            if (i.split('>')[0]==name):
                add=i.split('>')[1]
                cus_find=True

    
    
 
 # printing result
#print("Does date match format? : " + str(res))


    res =False
    while res==False:
   
        date=input("enter todays date")
        format = "%d-%m-%Y"
        try:
            res = bool(datetime.strptime(date, format))
        except ValueError:
            res = False
    
    
    sheet["H3"]=date
    sheet["H5"]=invoice_no
    cell_no=13
    add_list=add.split("|")
    for i in add_list:
        cell_str="B"+str(cell_no)
        sheet[cell_str]=i
        cell_str="E"+str(cell_no)
        sheet[cell_str]=i
        cell_no=cell_no+1
    
    sum_total=0
    status=False
    discription_details=[]
    HSN_dis={}
    GST_HSN={}
    
    pur_invoice_no=input("enter purchase bill no")
    pur_destination= r"C:\Users\AMARJEET\Desktop\python_bill\AUDIT\Audit"+pur_invoice_no+".xlsx"
    
        #load excel file
    workbook_pur = load_workbook(filename=pur_destination) 
    #open workbook
    sheet_pur = workbook_pur.active
    print(sheet_pur)
    net_amount=int(input("enter the amount"))
    net_amount_temp=0
    while status ==False:
        change_bill=int(input("press 1 to change_purchase_bill"))
        if change_bill==1:
            workbook_pur.save(filename=pur_destination)
            pur_invoice_no=input("enter purchase bill no")
            pur_destination= r"C:\Users\AMARJEET\Desktop\python_bill\AUDIT\Audit"+pur_invoice_no+".xlsx"
    
        #load excel file
            workbook_pur = load_workbook(filename=pur_destination)
 
        #open workbook
            sheet_pur = workbook_pur.active

        temp_list=[]
        pur_sl=int(input("enter serial no of product according to purchase bill"))
        cell_str="B"+str(5+pur_sl)
        try:
            discription=sheet_pur[cell_str].value
            print("product_discription:",discription)
        except Exception as Ex:
            print("Exception Occured during reading discription:",Ex)
            
        
        temp_list.append(discription)
        
        cell_str="G"+str(5+pur_sl)
        hsn=int(sheet_pur[cell_str].value)
        temp_list.append(hsn)
        if hsn in HSN_dis.keys():
            cc="sdddddd"
        else:
            HSN_dis[hsn]=0
        
        
        
        cell_str="V"+str(5+pur_sl)
        qty_rem=int(sheet_pur[cell_str].value)
        #net_amount=int(input("enter the amount"))
        if net_amount_temp ==0:
            net_amount_temp = net_amount
        if qty_rem ==0:
            print("this item is not in stock")
            continue
        print("quantity in stock for this product is",qty_rem)
        #full_qty=int(input("press 1 to use full qty"))
        #if full_qty==1:
         #   temp_quantity=qty_rem
        
        #else:
        #    temp_quantity=int(input("quantity"))
       # if temp_quantity>qty_rem:
         #   print("entered quantity is not avilable in stock")
       #     continue
        #temp_list.append(temp_quantity)
        cell_str="I"+str(5+pur_sl)
        rate=round(float(sheet_pur[cell_str].value),2)
        
        
        cell_str="J"+str(5+pur_sl)
        dis=round(float(sheet_pur[cell_str].value),2)
        
        cell_str="S"+str(5+pur_sl)
        gst=int(sheet_pur[cell_str].value)
        print("the value of gst is")
        temp_var =gst+100
        gst_factor =round(float(temp_var/100),2)
        print("the gst_factor is :",gst_factor)
        tax_amount =net_amount_temp/gst_factor
        print('taxable ammount is',tax_amount)
        
       
        rate=round((rate+(0.032*rate)),2)
        print('genral rate of the product is:',rate)
        rate_with_dis=round((rate-(rate*dis/100)),2)
        req_qnt=int((tax_amount/rate_with_dis))
        
        if req_qnt > qty_rem:
           # Temp_ammount = Temp_ammount +(req_qnt*rate_with_dis)+(req_qnt*rate_with_dis*gst/100)
            print("NOT SUFFICIENT NEED TO ADD MORE")
            rate = rate_with_dis 
            req_qnt = qty_rem
           # break
        
        else:
            rate=round((tax_amount/req_qnt),3)
            
            
        dis=0
        quantity= req_qnt
        temp_quantity=req_qnt
        
        
        temp_list.append(temp_quantity)
        temp_list.append(rate)
        print("optimum rate for product is ",rate)
        quantity=temp_quantity
        HSN_dis[hsn]=HSN_dis[hsn]+((quantity*rate)-(quantity*rate*dis/100))
        temp_list.append(dis)
        temp_list.append((quantity*rate)-(quantity*rate*dis/100))
        amount=(quantity*rate)-(quantity*rate*dis/100)
        
        
        cell_str="S"+str(5+pur_sl)
        gst=int(sheet_pur[cell_str].value)
        if hsn in GST_HSN.keys():
            cc="sdddddd"
        else:
            GST_HSN[hsn]=gst
        
        #gst=int(input("GST_rate"))
        sum_total=sum_total+(quantity*rate)+(quantity*rate*gst/100)
        temp_list.append(round((quantity*rate)+(quantity*rate*gst/100),2))
        discription_details.append(temp_list)
        print("total amount till now incluing gst",sum_total)
        print("NEED TO ADD MORE OF RS ",(net_amount -sum_total))
        
        net_amount_temp = net_amount -sum_total
        #inserting sell bill_no
        cell_str="L"+str(5+pur_sl)
        try:
            sell_bill_no=sheet_pur[cell_str].value
            sell_bill_no=sell_bill_no+'|'+str(invoice_no)
        except TypeError:
            sell_bill_no=str(invoice_no)
        sheet_pur[cell_str]=sell_bill_no
        #inserting sell_qty_wrt_bill
        cell_str="M"+str(5+pur_sl)
        try:
            sell_qty_no=sheet_pur[cell_str].value
            sell_qty_no=sell_qty_no+'|'+str(quantity)
        except TypeError:
            sell_qty_no=str(quantity)
        sheet_pur[cell_str]=sell_qty_no
        
        
        #inserting total_quantity_selled in audit
        
        cell_str="O"+str(5+pur_sl)
        try:
            total_selled_qntity=int(sheet_pur[cell_str].value)
        except TypeError:
            total_selled_qntity=0;
        
        total_selled_qntity=total_selled_qntity+quantity
        sheet_pur[cell_str]=str(total_selled_qntity)
        
        #inserting total selled qntity
        cell_str="O"+str(5+pur_sl)
        sheet_pur[cell_str]=total_selled_qntity
        cell_str="U"+str(5+pur_sl)
        sheet_pur[cell_str]=total_selled_qntity
        
        
        #inserting quantity in stock
        cell_str="H"+str(5+pur_sl)
        pur_total_qntity=int(sheet_pur[cell_str].value)
        stock_qnty=pur_total_qntity-total_selled_qntity
        cell_str="V"+str(5+pur_sl)
        sheet_pur[cell_str]=stock_qnty
        
        #inserting sell amount
        
        cell_str="X"+str(5+pur_sl)
        try:
            total_selled_amount=int(sheet_pur[cell_str].value)
        except TypeError:
            total_selled_amount=0
        total_selled_amount=total_selled_amount+amount
        sheet_pur[cell_str]=total_selled_amount
        
        cell_str="P"+str(5+pur_sl)
        sheet_pur[cell_str]= round( (float(total_selled_amount)/float(total_selled_qntity)),2)
   
        #inserting gst__received
        
        cell_str="Z"+str(5+pur_sl)
        try:
            gst_amount=float(round(sheet_pur[cell_str].value))
        except TypeError:
            gst_amount=0
        gst_amount=gst_amount+round((amount*gst/100),2)
        sheet_pur[cell_str]=str(gst_amount)
        
        #inserting gst__balance
        
        
        cell_str="Y"+str(5+pur_sl)
        try:
            gst_recv=int(sheet_pur[cell_str].value)
        except TypeError:
            gst_recv=0
        gst_balance=gst_amount-gst_recv
        cell_str="AA"+str(5+pur_sl)
        sheet_pur[cell_str]=str(gst_balance)
        
        
        x=int(input("press 1 to add more"))
        if(x==1):
              status=False
        else:
              status=True
    num=21
    count=0
    s_t=0
    print(discription_details)
    for i in discription_details:
        count=count+1
        cell_str="A"+str(num)
        sheet[cell_str]=count
        cell_str="B"+str(num)
        sheet[cell_str]=i[0]
        cell_str="E"+str(num)
        sheet[cell_str]=i[1]
        cell_str="F"+str(num)
        sheet[cell_str]=i[2]
        cell_str="G"+str(num)
        sheet[cell_str]=i[3]
        cell_str="H"+str(num)
        sheet[cell_str]=i[4]
        cell_str="I"+str(num)
        sheet[cell_str]=i[5]
        s_t=s_t+i[-2]
        num=num+1
    
    sheet["H27"]=s_t
    
    
    count=37
    tax_sum=0
    for key, value_1 in HSN_dis.items():
    
        cell_str="B"+str(count)
        sheet[cell_str]=key
        sheet.cell(count,3).value=value_1
    #cell_str="D"+str(count)
    #sheet[cell_str]=value_1
        cell_str="E"+str(count)
        sheet[cell_str]=(GST_HSN[key]/2)/100
        cell_str="F"+str(count)
        sheet[cell_str]=(GST_HSN[key]/2)*value_1/100
        cell_str="G"+str(count)
        sheet[cell_str]=(GST_HSN[key]/2)/100
        cell_str="H"+str(count)
        sheet[cell_str]=(GST_HSN[key]/2)*value_1/100
        cell_str="I"+str(count)
        sheet[cell_str]=(GST_HSN[key])*value_1/100
        tax_sum=tax_sum+(GST_HSN[key])*value_1/100
        count=count+1
    sheet["J41"]=tax_sum
    sheet["C42"]=num2words.num2words(round(tax_sum,2), lang='en_IN')+"ONLY"
    sheet["H28"]=tax_sum/2
    sheet["H29"]=tax_sum/2
    sheet["H30"]=s_t+tax_sum
    
    
    sum_t=s_t+tax_sum
    r_total=round(sum_t,0)
    sheet['H31']=net_amount-(s_t+tax_sum)
    sheet['H32']=net_amount
    sheet["C34"]=num2words.num2words(net_amount, lang='en_IN')+"ONLY"

#save the file
    workbook_pur.save(filename=pur_destination)
    workbook.save(filename=destination)
    workbook.close()

   # pdf_path=r"C:\Users\AMARJEET\Desktop\python_bill\SELL_BILL\PDF\AUG\sk"+invoice_no
    pdf_path1=r"C:\Users\AMARJEET\Desktop\python_bill\SELL_BILL\PDF\JUNE24\sk"+invoice_no
# Open  Excel
    excel = client.Dispatch("Excel.Application")

    # Read Excel File
    sheets = excel.Workbooks.Open(destination)
    work_sheets = sheets.Worksheets[0]
  
    # Convert into PDF File
    #work_sheets.ExportAsFixedFormat(0, pdf_path)
    work_sheets.ExportAsFixedFormat(0, pdf_path1)
    ##############COMMENTED FOR DISABLE PRINTING ###########################################
   # for i in range(1):    
    #    os.startfile(destination, "print")
     #   for p in psutil.process_iter(): #Close Acrobat after printing the PDF
      #      if 'Excel' in str(p):
       #         p.kill()
                
    

enter current invoice number266
enter the customer nameSF
enter todays date30-06-2024
enter purchase bill no2152
<Worksheet "Sheet1">
enter the amount18816
press 1 to change_purchase_bill0
enter serial no of product according to purchase bill1
product_discription: 72 PAGES JUMBO EX NOTE BOOK
quantity in stock for this product is 1090
the value of gst is
the gst_factor is : 1.12
taxable ammount is 16800.0
genral rate of the product is: 7.91
NOT SUFFICIENT NEED TO ADD MORE
optimum rate for product is  7.91
total amount till now incluing gst 9656.528
NEED TO ADD MORE OF RS  9159.472
press 1 to add more1
press 1 to change_purchase_bill0
enter serial no of product according to purchase bill2
product_discription: 172 PAGES JUMBO EX NOTE BOOK
quantity in stock for this product is 2047
the value of gst is
the gst_factor is : 1.12
taxable ammount is 8178.099999999999
genral rate of the product is: 14.61
optimum rate for product is  14.63
total amount till now incluing gst 18816.078400000002
NEE

In [ ]:
import os
pdf_path= r"C:\Users\HP\Desktop\python_bill\SELL_BILL\PDF\sk0288"
os.startfile(r'PIL.pdf','print')

In [2]:
import os
import shutil
import re
from openpyxl import load_workbook
from datetime import datetime
from win32com import client
import num2words
import psutil
from time import sleep
while(True):
    res =False
    while res ==False:
    
        invoice_no=input("enter current invoice number")
        pat = re.compile(r"[0-9]+")
    # Checks whether the whole string matches the re.pattern or not
        if re.fullmatch(pat, invoice_no):
            res=True
        else:
            print("not valid invoice number")
        





    source = r"C:\Users\AMARJEET\Desktop\python_bill\formate.xlsx"
    destination= r"C:\Users\AMARJEET\Desktop\python_bill\SELL_BILL\EXCEL\sk"+invoice_no+".xlsx"
    file_loc=r"C:\Users\AMARJEET\Desktop\python_bill\CUSTOMER.txt"
    
    

    dest = shutil.copyfile(source, destination)

#load excel file
    workbook = load_workbook(filename=destination)
 
    #open workbook
    sheet = workbook.active


    cos_file=open(file_loc)
    cus_details=cos_file.readlines()
    cus_find=False
    while cus_find==False:
        name=input("enter the customer name")
        list_len=len(cus_details)
        for i in cus_details:
            if (i.split('>')[0]==name):
                add=i.split('>')[1]
                cus_find=True

    
    
 
 # printing result
#print("Does date match format? : " + str(res))


    res =False
    while res==False:
   
        date=input("enter todays date")
        format = "%d-%m-%Y"
        try:
            res = bool(datetime.strptime(date, format))
        except ValueError:
            res = False
    
    
    sheet["H3"]=date
    sheet["H5"]=invoice_no
    cell_no=13
    add_list=add.split("|")
    for i in add_list:
        cell_str="B"+str(cell_no)
        sheet[cell_str]=i
        cell_str="E"+str(cell_no)
        sheet[cell_str]=i
        cell_no=cell_no+1
    
    sum_total=0
    status=False
    discription_details=[]
    HSN_dis={}
    GST_HSN={}
    
    pur_invoice_no=input("enter purchase bill no")
    pur_destination= r"C:\Users\AMARJEET\Desktop\python_bill\AUDIT\Audit"+pur_invoice_no+".xlsx"
    
        #load excel file
    workbook_pur = load_workbook(filename=pur_destination) 
    #open workbook
    sheet_pur = workbook_pur.active
    print(sheet_pur)
    while status ==False:
        change_bill=int(input("press 1 to change_purchase_bill"))
        if change_bill==1:
            workbook_pur.save(filename=pur_destination)
            pur_invoice_no=input("enter purchase bill no")
            pur_destination= r"C:\Users\AMARJEET\Desktop\python_bill\AUDIT\Audit"+pur_invoice_no+".xlsx"
    
        #load excel file
            workbook_pur = load_workbook(filename=pur_destination)
 
        #open workbook
            sheet_pur = workbook_pur.active

        temp_list=[]
        pur_sl=int(input("enter serial no of product according to purchase bill"))
        cell_str="B"+str(5+pur_sl)
        try:
            discription=sheet_pur[cell_str].value
            print("product_discription:",discription)
        except Exception as Ex:
            print("Exception Occured during reading discription:",Ex)
            
        
        temp_list.append(discription)
        
        cell_str="G"+str(5+pur_sl)
        hsn=int(sheet_pur[cell_str].value)
        temp_list.append(hsn)
        if hsn in HSN_dis.keys():
            cc="sdddddd"
        else:
            HSN_dis[hsn]=0
        
        
        
        cell_str="V"+str(5+pur_sl)
        qty_rem=int(sheet_pur[cell_str].value)
        net_amount=int(input("enter the amount"))
        if qty_rem ==0:
            print("this item is not in stock")
            continue
        print("quantity in stock for this product is",qty_rem)
        #full_qty=int(input("press 1 to use full qty"))
        #if full_qty==1:
         #   temp_quantity=qty_rem
        
        #else:
        #    temp_quantity=int(input("quantity"))
       # if temp_quantity>qty_rem:
         #   print("entered quantity is not avilable in stock")
       #     continue
        #temp_list.append(temp_quantity)
        cell_str="I"+str(5+pur_sl)
        rate=round(float(sheet_pur[cell_str].value),2)
        
        
        cell_str="J"+str(5+pur_sl)
        dis=round(float(sheet_pur[cell_str].value),2)
        
        cell_str="S"+str(5+pur_sl)
        gst=int(sheet_pur[cell_str].value)
        print("the value of gst is")
        temp_var =gst+100
        gst_factor =round(float(temp_var/100),2)
        print("the gst_factor is :",gst_factor)
        tax_amount =net_amount/gst_factor
        print('taxable ammount is',tax_amount)
        
       
        rate=round((rate+(0.032*rate)),2)
        print('genral rate of the product is:',rate)
        rate_with_dis=round((rate-(rate*dis/100)),2)
        req_qnt=int((tax_amount/rate_with_dis))
        
        if req_qnt > qty_rem:
            Temp_ammount = Temp_ammount +(req_qnt*rate_with_dis)+(req_qnt*rate_with_dis*gst/100)
            print("the total ammount till now is ",Temp_ammount)
            break
        
        else:
            rate=round((tax_amount/req_qnt),3)
            
            
        dis=0
        quantity= req_qnt
        temp_quantity=req_qnt
        
        
        temp_list.append(temp_quantity)
        temp_list.append(rate)
        print("optimum rate for product is ",rate)
        quantity=temp_quantity
        HSN_dis[hsn]=HSN_dis[hsn]+((quantity*rate)-(quantity*rate*dis/100))
        temp_list.append(dis)
        temp_list.append((quantity*rate)-(quantity*rate*dis/100))
        amount=(quantity*rate)-(quantity*rate*dis/100)
        
        
        cell_str="S"+str(5+pur_sl)
        gst=int(sheet_pur[cell_str].value)
        if hsn in GST_HSN.keys():
            cc="sdddddd"
        else:
            GST_HSN[hsn]=gst
        
        #gst=int(input("GST_rate"))
        sum_total=sum_total+(quantity*rate)+(quantity*rate*gst/100)
        temp_list.append(round((quantity*rate)+(quantity*rate*gst/100),2))
        discription_details.append(temp_list)
        print("total amount till now incluing gst",sum_total)
        
        #inserting sell bill_no
        cell_str="L"+str(5+pur_sl)
        try:
            sell_bill_no=sheet_pur[cell_str].value
            sell_bill_no=sell_bill_no+'|'+str(invoice_no)
        except TypeError:
            sell_bill_no=str(invoice_no)
        sheet_pur[cell_str]=sell_bill_no
        #inserting sell_qty_wrt_bill
        cell_str="M"+str(5+pur_sl)
        try:
            sell_qty_no=sheet_pur[cell_str].value
            sell_qty_no=sell_qty_no+'|'+str(quantity)
        except TypeError:
            sell_qty_no=str(quantity)
        sheet_pur[cell_str]=sell_qty_no
        
        
        #inserting total_quantity_selled in audit
        
        cell_str="O"+str(5+pur_sl)
        try:
            total_selled_qntity=int(sheet_pur[cell_str].value)
        except TypeError:
            total_selled_qntity=0;
        
        total_selled_qntity=total_selled_qntity+quantity
        sheet_pur[cell_str]=str(total_selled_qntity)
        
        #inserting total selled qntity
        cell_str="O"+str(5+pur_sl)
        sheet_pur[cell_str]=total_selled_qntity
        cell_str="U"+str(5+pur_sl)
        sheet_pur[cell_str]=total_selled_qntity
        
        
        #inserting quantity in stock
        cell_str="H"+str(5+pur_sl)
        pur_total_qntity=int(sheet_pur[cell_str].value)
        stock_qnty=pur_total_qntity-total_selled_qntity
        cell_str="V"+str(5+pur_sl)
        sheet_pur[cell_str]=stock_qnty
        
        #inserting sell amount
        
        cell_str="X"+str(5+pur_sl)
        try:
            total_selled_amount=int(sheet_pur[cell_str].value)
        except TypeError:
            total_selled_amount=0
        total_selled_amount=total_selled_amount+amount
        sheet_pur[cell_str]=total_selled_amount
        
        cell_str="P"+str(5+pur_sl)
        sheet_pur[cell_str]= round( (float(total_selled_amount)/float(total_selled_qntity)),2)
   
        #inserting gst__received
        
        cell_str="Z"+str(5+pur_sl)
        try:
            gst_amount=float(round(sheet_pur[cell_str].value))
        except TypeError:
            gst_amount=0
        gst_amount=gst_amount+round((amount*gst/100),2)
        sheet_pur[cell_str]=str(gst_amount)
        
        #inserting gst__balance
        
        
        cell_str="Y"+str(5+pur_sl)
        try:
            gst_recv=int(sheet_pur[cell_str].value)
        except TypeError:
            gst_recv=0
        gst_balance=gst_amount-gst_recv
        cell_str="AA"+str(5+pur_sl)
        sheet_pur[cell_str]=str(gst_balance)
        
        
        x=int(input("press 1 to add more"))
        if(x==1):
              status=False
        else:
              status=True
    num=21
    count=0
    s_t=0
    print(discription_details)
    for i in discription_details:
        count=count+1
        cell_str="A"+str(num)
        sheet[cell_str]=count
        cell_str="B"+str(num)
        sheet[cell_str]=i[0]
        cell_str="E"+str(num)
        sheet[cell_str]=i[1]
        cell_str="F"+str(num)
        sheet[cell_str]=i[2]
        cell_str="G"+str(num)
        sheet[cell_str]=i[3]
        cell_str="H"+str(num)
        sheet[cell_str]=i[4]
        cell_str="I"+str(num)
        sheet[cell_str]=i[5]
        s_t=s_t+i[-2]
        num=num+1
    
    sheet["H27"]=s_t
    
    
    count=37
    tax_sum=0
    for key, value_1 in HSN_dis.items():
    
        cell_str="B"+str(count)
        sheet[cell_str]=key
        sheet.cell(count,3).value=value_1
    #cell_str="D"+str(count)
    #sheet[cell_str]=value_1
        cell_str="E"+str(count)
        sheet[cell_str]=(GST_HSN[key]/2)/100
        cell_str="F"+str(count)
        sheet[cell_str]=(GST_HSN[key]/2)*value_1/100
        cell_str="G"+str(count)
        sheet[cell_str]=(GST_HSN[key]/2)/100
        cell_str="H"+str(count)
        sheet[cell_str]=(GST_HSN[key]/2)*value_1/100
        cell_str="I"+str(count)
        sheet[cell_str]=(GST_HSN[key])*value_1/100
        tax_sum=tax_sum+(GST_HSN[key])*value_1/100
        count=count+1
    sheet["J41"]=tax_sum
    sheet["C42"]=num2words.num2words(round(tax_sum,2), lang='en_IN')+"ONLY"
    sheet["H28"]=tax_sum/2
    sheet["H29"]=tax_sum/2
    sheet["H30"]=s_t+tax_sum
    
    
    sum_t=s_t+tax_sum
    r_total=round(sum_t,0)
    sheet['H31']=net_amount-sum_t
    sheet['H32']=net_amount
    sheet["C34"]=num2words.num2words(net_amount, lang='en_IN')+"ONLY"

#save the file
    workbook_pur.save(filename=pur_destination)
    workbook.save(filename=destination)
    workbook.close()

    pdf_path=r"C:\Users\AMARJEET\Desktop\python_bill\SELL_BILL\PDF\sk"+invoice_no
# Open  Excel
    excel = client.Dispatch("Excel.Application")

    # Read Excel File
    sheets = excel.Workbooks.Open(destination)
    work_sheets = sheets.Worksheets[0]


    # Convert into PDF File
    work_sheets.ExportAsFixedFormat(0, pdf_path)
    for i in range(1):    
        os.startfile(destination, "print")
        for p in psutil.process_iter(): #Close Acrobat after printing the PDF
            if 'Excel' in str(p):
                p.kill()
                
    

enter current invoice numberREEGAL929
not valid invoice number
enter current invoice number1319
enter the customer nameTAGORE
enter todays date04-12-2023
enter purchase bill noREEGAL929
<Worksheet "Sheet1">
press 1 to change_purchase_bill1
enter purchase bill noREEGAL929
enter serial no of product according to purchase bill20
product_discription: 172 A/4 Exercise Note Book
enter the amount1360
quantity in stock for this product is 40
the value of gst is
the gst_factor is : 1.12
taxable ammount is 1214.2857142857142
genral rate of the product is: 26.83


NameError: name 'Temp_ammount' is not defined

In [ ]:
BHAVESH2624